In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle

In [ ]:
text=pd.read_csv("mb_data_2.csv" ,index_col='type',engine='python', error_bad_lines=False)
print(text.shape)
print(text[0:5])
#print(text.iloc[2])

<ipython-input-2-53e2cfee5d48>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  text=pd.read_csv("mb_data_2.csv" ,index_col='type',engine='python', error_bad_lines=False)


FileNotFoundError: ignored

In [ ]:
from sklearn.preprocessing import LabelBinarizer

# One hot encode labels
labels=text.index.tolist()
print(labels[50:55])
encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)
labels=encoder.fit_transform(labels)
labels=np.array(labels)
print(labels[50:55])

In [ ]:
mbti_dict={0:'ENFJ',1:'ENFP',2:'ENTJ',3:'ENTP',4:'ESFJ',5:'ESFP',6:'ESTJ',7:'ESTP',8:'INFJ',9:'INFP',10:'INTJ',11:'INTP',12:'ISFJ',13:'ISFP',14:'ISFP',15:'ISTP'}

In [ ]:
import re

# Function to clean data..
def post_cleaner(post):
    """cleans individual posts`.
    Args:
        post-string
    Returns:
         cleaned up post`.
    """
    # Covert all uppercase characters to lower case
    post = post.lower() 
    
    # Remove |||
    post=post.replace('|||',"") 

    # Remove URLs, links etc
    post = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', '', post, flags=re.MULTILINE) 
    # This would have removed most of the links but probably not all 

    # Remove puntuations 
    puncs1=['@','#','$','%','^','&','*','(',')','-','_','+','=','{','}','[',']','|','\\','"',"'",';',':','<','>','/']
    for punc in puncs1:
        post=post.replace(punc,'') 

    puncs2=[',','.','?','!','\n']
    for punc in puncs2:
        post=post.replace(punc,' ') 
    # Remove extra white spaces
    post=re.sub( '\s+', ' ', post ).strip()
    return post

In [ ]:
posts=text.posts.tolist()
posts=[post_cleaner(post) for post in posts]

In [ ]:
from collections import Counter

word_count=Counter()
for post in posts:
    word_count.update(post.split(" "))
    
vocab_len=len(word_count)
print(vocab_len)

In [ ]:
vocab = sorted(word_count, key=word_count.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}
posts_ints=[]

for post in posts:
    posts_ints.append([vocab_to_int[word] for word in post.split()])

In [ ]:
print(posts_ints[0][:10])
print(len(posts_ints[0]))

In [ ]:
posts_lens = Counter([len(x) for x in posts])
print(posts_lens)
print(#np.mean(posts_lens.keys()) #(0.5*np.std(posts_lens.keys()))
)

In [ ]:
seq_len = 1000
features=np.zeros((len(posts_ints),seq_len),dtype=int)
for i, row in enumerate(posts_ints):
    features[i, -len(row):] = np.array(row)[:seq_len]
#print(features[1555])

In [ ]:
split_frac = 0.8

num_ele=int(split_frac*len(features))
rem_ele=len(features)-num_ele
train_x, val_x = features[:num_ele],features[num_ele: num_ele + int(rem_ele/2)]
train_y, val_y = labels[:num_ele],labels[num_ele:num_ele + int(rem_ele/2)]

test_x =features[num_ele + int(rem_ele/2):]
test_y = labels[num_ele + int(rem_ele/2):]

In [ ]:
lstm_size = 256
lstm_layers = 1
batch_size = 256
learning_rate = 0.01
embed_dim=250
n_words = len(vocab_to_int) + 1

In [ ]:
!pip install opendatasets
import opendatasets as od

od.download("https://www.kaggle.com/datasets/leadbest/googlenewsvectorsnegative300")

In [ ]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
int_to_vocab = {ii: word for word,ii in vocab_to_int.items()}

In [ ]:
#model[int_to_vocab[2]]

In [ ]:
wv_emb = []
unk=[]
for k in range(1,len(int_to_vocab)):
    try:
        wv_emb.append(model[int_to_vocab[k]])
    except:
        unk.append(int_to_vocab[k])
len(unk)

In [ ]:
unk[160:170]

In [ ]:
import tensorflow as tf
graph = tf.Graph()
with graph.as_default():
    input_data = tf.compat.v1.placeholder(tf.int32, [None, None], name='inputs')
    labels_ = tf.compat.v1.placeholder(tf.int32, [None, None], name='labels')
    keep_prob = tf.compat.v1.placeholder(tf.float32, name='keep_prob')
    
    embedding= tf.compat.v1.Variable(tf.random.uniform(shape=(n_words,embed_dim),minval=-1,maxval=1))
    embed=tf.compat.v1.nn.embedding_lookup(embedding,input_data)
    #print(embed.shape)
    
    lstm = tf.compat.v1.nn.rnn_cell.BasicLSTMCell(lstm_size)
    drop = tf.compat.v1.nn.rnn_cell.DropoutWrapper(lstm,output_keep_prob=keep_prob)
    cell = tf.compat.v1.nn.rnn_cell.MultiRNNCell([drop]* lstm_layers)
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    outputs,final_state=tf.compat.v1.nn.dynamic_rnn(cell,embed,dtype=tf.float32 )
    pre = tf.compat.v1.layers.dense(outputs[:,-1], 16, activation=tf.nn.relu)
    predictions=tf.compat.v1.layers.dense(pre, 16, activation=tf.nn.softmax)
    
    cost = tf.compat.v1.losses.mean_squared_error(labels_, predictions)
    optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate).minimize(cost)
    
    correct_pred = tf.compat.v1.equal(tf.cast(tf.round(predictions), tf.int32), labels_)
    accuracy = tf.compat.v1.reduce_mean(tf.cast(correct_pred, tf.float32))

    

In [ ]:
def get_batches(x, y, batch_size=100):
    
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

In [ ]:
epochs = 3

with graph.as_default():
    saver = tf.compat.v1.train.Saver()

with tf.compat.v1.Session(graph=graph) as sess:
    sess.run(tf.compat.v1.global_variables_initializer())
    iteration = 1
    for e in range(epochs):
        state = sess.run(initial_state)
        
        for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
            feed = {input_data: x,
                    labels_: y,
                    keep_prob: 1.0,
                    initial_state: state}
            loss, state, _ = sess.run([cost, final_state, optimizer], feed_dict=feed)
            
            if iteration%5==0:
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(loss))

            if iteration%25==0:
                val_acc = []
                val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                for x, y in get_batches(val_x, val_y, batch_size):
                    feed = {input_data: x,
                            labels_: y,
                            keep_prob: 1,
                            initial_state: val_state}
                    batch_acc, val_state = sess.run([accuracy, final_state], feed_dict=feed)
                    val_acc.append(batch_acc)
                print("Val acc: {:.3f}".format(np.mean(val_acc)))
            iteration +=1
    saver.save(sess, "checkpoints/mbti.ckpt")
    

In [ ]:
#y_test_pr=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
temp=0
f1=0
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)

#y_test_predictions=np.array(y_test_pr)
test_acc = []
with tf.compat.v1.Session(graph=graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    test_state = sess.run(cell.zero_state(batch_size, tf.float32))
    for ii, (x, y) in enumerate(get_batches(test_x, test_y, batch_size), 1):
        feed = {input_data: x,
                labels_: y,
                keep_prob: 1,
                initial_state: test_state}
        #print("hey")
        batch_acc, test_state = sess.run([accuracy, final_state], feed_dict=feed)
        temp+=sess.run(predictions,feed_dict=feed).shape[0]
        y_test_predictions=sess.run(predictions,feed_dict=feed)
        #y_test_predictions.flatten()
        if y_test_predictions.any():
          y_test_predictions = np.hstack(y_test_predictions)
        else:
          y_test_predictions = y_test_predictions.flatten()
        #y_test_predictionss= np.where(y_test_predictions==max(y_test_predictions),1,0)
        print(y_test_predictions)
        if y.any():
          y = np.hstack(y)
        else:
          y = y.flatten()
        for i in range(0,y_test_predictions.shape[0]):
          j=0
          mx=0
          index=0
          while j<16 and i+j<4096:
            if y_test_predictions[i+j]>mx:
              mx=y_test_predictions[i+j]
              index=i
            j+=1
          j=0
          while j<16 and i+j<4096:
            if y_test_predictions[i+j]==mx:
              y_test_predictions[i+j]=1
            else:
              y_test_predictions[i+j]=0
            
            j+=1
          i=i+15
          
          
        print(y.shape)
        print(temp)
        f1=f1_score(y_true=y, y_pred=y_test_predictions, average='weighted')
        #y_test_predictions=np.concatenate(( y_test_predictions,sess.run(predictions,feed_dict=feed)),axis=0)
        test_acc.append(batch_acc)
   # print(test_acc)
   # print(y_test_predictions)
    print("Test accuracy: {:.5f}".format(np.mean(test_acc)))
    print("F1 scoree: {:.5f}".format(f1))

In [ ]:
tempp=text.index.tolist()
encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)
names_encode = encoder.fit(tempp)


In [ ]:
#y_test_pr=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
temp=0
f1=0
re=0
pr=0
ac=0
Label1pred=[]
Label2pred=[]
Label3pred=[]
Label4pred=[]
Label1true=[]
Label2true=[]
Label3true=[]
Label4true=[]

from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)

#y_test_predictions=np.array(y_test_pr)
test_acc = []
f1_total = []
recall_total = []
prec_total = []
acc_total = []
with tf.compat.v1.Session(graph=graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    test_state = sess.run(cell.zero_state(batch_size, tf.float32))
    for ii, (x, y) in enumerate(get_batches(test_x, test_y, batch_size), 1):
        feed = {input_data: x,
                labels_: y,
                keep_prob: 1,
                initial_state: test_state}
        #print("hey")
        batch_acc, test_state = sess.run([accuracy, final_state], feed_dict=feed)
        temp+=sess.run(predictions,feed_dict=feed).shape[0]
        y_test_predictions=sess.run(predictions,feed_dict=feed)
        #confusion matrix
        y_testing=sess.run(correct_pred,feed_dict=feed)
        true_class = tf.compat.v1.argmax( y, 1 )
        predicted_class = tf.compat.v1.argmax( y_testing, 1 )
        #print(true_class[:5])
        #print(predicted_class[:5])
        cm_running_total = None
        cm_nupmy_array = sess.run(tf.compat.v1.confusion_matrix(true_class, predicted_class, 16), feed_dict=feed )
        ##print(sess.run(tf.compat.v1.confusion_matrix(true_class, predicted_class, 16), feed_dict=feed ))
        




        backlabel=names_encode.inverse_transform(y_test_predictions)

        truebacklabel=names_encode.inverse_transform(y)
        for i in range(0,y_test_predictions.shape[0]):
          j=0
          mx=0
          index=0
          while j<16:
            if y_test_predictions[i][j]>mx:
              mx=y_test_predictions[i][j]
              index=i
              #print("Hey")
            j+=1
          j=0
          while j<16:
            if y_test_predictions[i][j]==mx:
              y_test_predictions[i][j]=1
            else:
              y_test_predictions[i][j]=0
              #print("Hey2")
            j+=1
          j=0
          i+=15
        backlabel=names_encode.inverse_transform(y_test_predictions)
        y_test_predictions = y_test_predictions.astype('int32')
        
        #print(y_test_predictions[:5])
        #print(type(y_test_predictions))
        #print(y[:5])
        #print(backlabel[:5])
        #print(truebacklabel[:5])
        #y_test_predictions.flatten()
        """"if y_test_predictions.any():
          y_test_predictions = np.hstack(y_test_predictions)
        else:
          y_test_predictions = y_test_predictions.flatten()
        #y_test_predictionss= np.where(y_test_predictions==max(y_test_predictions),1,0)
        print(y_test_predictions)
        if y.any():
          y = np.hstack(y)
        else:
          y = y.flatten()
        for i in range(0,y_test_predictions.shape[0]):
          j=0
          mx=0
          index=0
          while j<16 and i+j<y_test_predictions.shape[0]:
            if y_test_predictions[i+j]>mx:
              mx=y_test_predictions[i+j]
              index=i
            j+=1
          j=0
          while j<16 and i+j<y_test_predictions.shape[0]:
            if y_test_predictions[i+j]==mx:
              y_test_predictions[i+j]=1
            else:
              y_test_predictions[i+j]=0
            
            j+=1
          i=i+15
        """
        
        for i in range(0,backlabel.shape[0]):
          if(truebacklabel[i][0]=='I'):
            Label1true.append(1)
          else:
            Label1true.append(0)
          if(truebacklabel[i][1]=='N'):
            Label2true.append(1)
          else:
            Label2true.append(0)
          
          if(truebacklabel[i][2]=='F'):
            Label3true.append(1)
          else:
            Label3true.append(0)
          
          if(truebacklabel[i][3]=='J'):
            Label4true.append(1)
          else:
            Label4true.append(0)
          
          if(backlabel[i][0]=='I'):
            Label1pred.append(1)
          else:
            Label1pred.append(0)

          if(backlabel[i][1]=='N'):
            Label2pred.append(1)
          else:
            Label2pred.append(0)
          
          if(backlabel[i][2]=='F'):
            Label3pred.append(1)
          else:
            Label3pred.append(0)
          
          if(backlabel[i][3]=='J'):
            Label4pred.append(1)
          else:
            Label4pred.append(0)
          
          
        print(Label4pred[:5])
        print(Label4true[:5])
        f1=f1_score(y_true=y, y_pred=y_test_predictions, average='weighted')
        #set() - set(y_test_predictions)
        re=recall_score(y_true=y, y_pred=y_test_predictions, average='weighted')
        pr=precision_score(y_true=y, y_pred=y_test_predictions, average='weighted')
        test_acc.append(batch_acc)
        f1_total.append(f1)
        recall_total.append(re)
        prec_total.append(pr)
        acc_total.append(ac)
        f1=f1_score(y_true=y, y_pred=y_test_predictions, average='weighted')
        re=recall_score(y_true=y, y_pred=y_test_predictions, average='weighted')
        pr=precision_score(y_true=y, y_pred=y_test_predictions, average='weighted')
     
   # print(test_acc)
   # print(y_test_predictions)
    #print("Class1: ")
    f1=f1_score(y_true=Label1true, y_pred=Label1pred)
    print("Class 1 F1 score: {:.5f}".format(f1))
    re=recall_score(y_true=Label1true, y_pred=Label1pred)
    pr=precision_score(y_true=Label1true, y_pred=Label1pred)
    ac=accuracy_score(y_true=Label1true, y_pred=Label1pred)
    print("Recall score: {:.5f}".format(re))
    print("Precision score: {:.5f}".format(pr))
    print("Accuracy score: {:.5f}".format(ac))
    print("Class2: ")
    f1=f1_score(y_true=Label2true, y_pred=Label2pred, average='weighted')
    re=recall_score(y_true=Label2true, y_pred=Label2pred)
    pr=precision_score(y_true=Label2true, y_pred=Label2pred)
    ac=accuracy_score(y_true=Label2true, y_pred=Label2pred)
    
    print("F1 score: {:.5f}".format(f1))
    print("Recall score: {:.5f}".format(re))
    print("Precision score: {:.5f}".format(pr))
    print("Accuracy score: {:.5f}".format(ac))
    #f1=f1_score(y_true=Label3true, y_pred=Label3pred, average='weighted')
    print("Class3: ")
    f1=f1_score(y_true=Label3true, y_pred=Label3pred, average='weighted')
    re=recall_score(y_true=Label3true, y_pred=Label3pred)
    pr=precision_score(y_true=Label3true, y_pred=Label3pred)
    ac=accuracy_score(y_true=Label3true, y_pred=Label3pred)
    print("F1 score: {:.5f}".format(f1))
    print("Recall score: {:.5f}".format(re))
    print("Precision score: {:.5f}".format(pr))
    print("Accuracy score: {:.5f}".format(ac))
    print("Class4: ")
    f1=f1_score(y_true=Label4true, y_pred=Label4pred, average='weighted')
    re=recall_score(y_true=Label4true, y_pred=Label4pred)
    pr=precision_score(y_true=Label4true, y_pred=Label4pred)
    
    print("F1 score: {:.5f}".format(f1))
    print("Recall score: {:.5f}".format(re))
    print("Precision score: {:.5f}".format(pr))
    print("Accuracy score: {:.5f}".format(ac))
   
    #print("F1 score: {:.5f}".format(np.mean(f1_total)))
    print("Test accuracy: {:.5f}".format(np.mean(test_acc)))
    #print("Recall: {:.5f}".format(np.mean(recall_total)))
    #print("Precision: {:.5f}".format(np.mean(prec_total)))
    
    #f1=(2*pr*re)/(pr+re)
    #print(f1)

In [ ]:
y_test_predictions.shape

In [ ]:
temp

In [ ]:
true_pos
for i in range(0,y_test_predictions.shape[0]):
  for j in (0,y_test_predictions.shape[1]):
    

In [ ]:
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
test_y.shape
np.concatenate((test_y,y_test_predictions),axis=0)
test_y.shape
#test_x.shape

In [ ]:
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)

# Generate confusion matrix for the predictions
conf_matrix = confusion_matrix(test_y, y_test_predictions)
conf_matrix